In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import neurokit2 as nk
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import AdaBoostClassifier
import pyhrv.time_domain as td
import pywt
import matplotlib.pyplot as plt
import pickle
import pyhrv
import ipynb
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# functions for features extraction
from ipynb.fs.full.ECG_features import get_ecgfeatures
from ipynb.fs.full.EDA import get_edaindex, get_edafeatures
from ipynb.fs.full.BVP import get_bvpfeatures, bvp_prep, sensorPPGfilter

c:\Users\riche\AppData\Local\Programs\Python\Python39\lib\site-packages\antropy\fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit('float64(float64[:], int32)')


# **Data Collection & Feature extraction**

In [2]:
# Dataset reading
class read_data_of_one_subject:
            """Read data from WESAD dataset"""
            def __init__(self, path, subject):
                self.keys = ['label', 'subject', 'signal']
                self.signal_keys = ['wrist', 'chest']
                self.chest_sensor_keys = ['ACC', 'ECG', 'EDA', 'EMG', 'Resp', 'Temp']
                self.wrist_sensor_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
                #os.chdir(path)
                #os.chdir(subject)
                with open(path + subject +'/'+subject + '.pkl', 'rb') as file:
                    data = pickle.load(file, encoding='latin1')
                self.data = data

            def get_labels(self):
                return self.data[self.keys[0]]

            def get_wrist_data(self):
                """"""
                #label = self.data[self.keys[0]]
                #assert subject == self.data[self.keys[1]]
                signal = self.data[self.keys[2]]
                wrist_data = signal[self.signal_keys[0]]
                #wrist_ACC = wrist_data[self.wrist_sensor_keys[0]]
                #wrist_ECG = wrist_data[self.wrist_sensor_keys[1]]
                return wrist_data

            def get_chest_data(self):
                """"""
                signal = self.data[self.keys[2]]
                chest_data = signal[self.signal_keys[1]]
                return chest_data

In [3]:
# Set up empty dataframes for the features
columns_ecg =['index', 'ecg_HR_mean', 'ecg_HR_min', 'ecg_HR_max', 'ecg_HR_std', 'ecg_SDNN', 'ecg_SDANN', 'ecg_RMSSD', 
                                           'ecg_SDSD','ecg_pNN50', 'ecg_pNN20', "ecg_triangular_index", "ecg_tinn", "ecg_sd1", "ecg_sd2",
                                            "ecg_ratio_sd2_sd1", 'ecg_abs_power_VLF', 'ecg_abs_power_LF', 'ecg_abs_power_HF', 'ecg_tot_power',
                                            'ecg_LF/HF', 'ecg_peak_vlf', 'ecg_peak_lf', 'ecg_peak_hf', 'ecg_norm_power_LF', 'ecg_norm_power_HF',
                                            'max_cwt_absmean', 'min_cwt_absmean', 'mean_cwt_absmean','std_cwt_absmean', 
                                            'max_cwt_std', 'min_cwt_std', 'mean_cwt_std','std_cwt_std',
                                            'max_cwt_energy', 'min_cwt_energy', 'mean_cwt_energy','std_cwt_energy',
                                            'max_cwt_var', 'min_cwt_var', 'mean_cwt_var','std_cwt_var',
                                            'entropy']
columns_resp = ['RSP_Rate_Mean','RRV_RMSSD', 'RRV_MeanBB', 'RRV_SDBB', 'RRV_SDSD', 'RRV_CVBB', 'RRV_CVSD', 'RRV_MedianBB',
                'RRV_MadBB', 'RRV_MCVBB', 'RRV_VLF', 'RRV_LF', 'RRV_HF', 'RRV_LFHF', 'RRV_LFn', 'RRV_HFn', 'RRV_SD1',
                'RRV_SD2', 'RRV_SD2SD1', 'RRV_ApEn', 'RRV_SampEn', 'RSP_Amplitude_Mean', 'RSP_RVT', 'RSP_Symmetry_PeakTrough',
                'RSP_Symmetry_RiseDecay', 'RSP_Phase_Duration_Inspiration', 'RSP_Phase_Duration_Expiration', 'RSP_Phase_Duration_Ratio']

columns = [*columns_ecg, *columns_resp]
ecgfeatures = pd.DataFrame(None, columns= columns)
edafeatures = None
bvpfeatures = None

print(len(ecgfeatures.keys()))
total_samples = 0
fs = 700 # sampling frequency
fs_bvp = 64
duration = 45

71


In [4]:
y = []
amountpsample = dict()
# data_set_path= "C:/Users/JackC/Documents/EPO4/WESAD/WESAD/" # Folder path
data_set_path= "C:/Users/riche/Downloads/WESAD/WESAD/" # Folder path
# data_set_path = "C:/Users/Adnane/Downloads/WESAD/WESAD/"

for i in range(16): # 15 subjects
    subject = 'S'+str(i+2) # Cycle through S2 to S17
    print(f"subject: {subject}")
    amountpsample[subject] = 0
    if subject != 'S12': # Skip S12, because it does not exist
        # Object instantiation
        obj_data = {}

        # Accessing class attributes and method through objects
        obj_data[subject] = read_data_of_one_subject(data_set_path, subject)

        chest_data_dict = obj_data[subject].get_chest_data()
        wrist_data_dict = obj_data[subject].get_wrist_data()
        chest_dict_length = {key: len(value) for key, value in chest_data_dict.items()}
        wrist_dict_length = {key: len(value) for key, value in wrist_data_dict.items()}

        # Get labels
        labels = obj_data[subject].get_labels()

        for label in [1,2,4]: # for every state of stress
            
            baseline = np.asarray([idx for idx,val in enumerate(labels) if val == label])

            # Obtaining the chest data
            eda_base=chest_data_dict['EDA'][baseline,0] # Select the EDA data
            ecg_base=chest_data_dict['ECG'][baseline,0] # Select the ECG data
            resp_base=chest_data_dict['Resp'][baseline,0] # Select the respiration data

            # Obtaining the wrist data
            baseline_BVP = baseline * fs_bvp // fs
            bvp_base=wrist_data_dict['BVP'][baseline_BVP,0] # Select the BVP data

            # Multiple smaller samples per signal
            # samples = 10 # specify amount of samples

            for j in range(0, len(eda_base)//(duration*fs), 2): # loop over all segments
                amountpsample[subject] += 1
                # adding the labels to y
                if label==1 or label==4: # if baseline or medidation, classify as non-stress
                    y.append(0)
                else:
                    y.append(1)

                # Cutting the signal into a segment
                start, end = j*fs*duration, (j+1)*fs*duration
                ecg = ecg_base[start: end]
                eda = eda_base[start: end]
                bvp = bvp_base[start: end]

                # splitting sample into 10 smaller even-sized parts
                #ecg = ecg_base[int(len(ecg_base)* (j/samples)) : int(len(ecg_base)* ((j+1)/samples))]
                #eda = eda_base[int(len(eda_base)* (j/samples)) : int(len(eda_base)* ((j+1)/samples))]

                # getting the eda and ecg features
                index = subject + str(label) + str(j)
                tonic, phasic, start, end = get_edaindex(eda, fs)
                edafeatures = get_edafeatures(index, edafeatures, phasic, tonic, fs)
                ecgfeatures = get_ecgfeatures(ecg, fs, ecgfeatures, index)
                
                bvpfeature = get_bvpfeatures(bvp, index)
                bvpfeatures = pd.concat([bvpfeatures, bvpfeature], axis=0)
    print(f"total samples {subject}: {amountpsample[subject]}")

subject: S2
total samples S2: 29
subject: S3
total samples S3: 29
subject: S4
total samples S4: 29
subject: S5
total samples S5: 29
subject: S6
total samples S6: 29
subject: S7
total samples S7: 29
subject: S8
total samples S8: 29
subject: S9
total samples S9: 29
subject: S10
total samples S10: 30
subject: S11
total samples S11: 30
subject: S12
total samples S12: 0
subject: S13
total samples S13: 29
subject: S14
total samples S14: 30
subject: S15
total samples S15: 30
subject: S16
total samples S16: 29
subject: S17
total samples S17: 29


In [5]:
# saved_edafeatures = edafeatures
# saved_ecgfeatures = ecgfeatures
# saved_bvpfeatures = bvpfeatures

In [11]:
bvp_filt = bvp_prep(bvpfeatures) # remove nan and inf in BVP features
# ecgfeatures = bvp_prep(ecgfeatures)

# saving the feature data in csv files
edafeatures.to_csv("EDA_features.csv")
ecgfeatures.to_csv("ECG_features.csv")
bvp_filt.to_csv("BVP_features.csv")

In [4]:
# #reading from csv file if necessary
# edafeatures = pd.read_csv("EDA_features.csv", index_col=0)
# ecgfeatures = pd.read_csv("ECG_features.csv", index_col=0)
# bvp_filt = pd.read_csv("BVP_features.csv", index_col=0)
#features = pd.read_csv("features_"+str(samples)+"_samp.csv", index_col=0)

In [12]:
features1 = pd.merge(ecgfeatures, edafeatures, left_index=True, right_index=True)
total_features = pd.merge(bvp_filt, features1, on='index')
total_features['y'] = y

total_features = total_features[total_features.RSP_RVT != 0.0]
total_features.to_csv("out_features.csv")

In [139]:
# saved_total_features = total_features
# total_features = pd.read_csv("out_features.csv", index_col=0)

In [140]:
total_features = total_features.fillna(method="ffill")
total_features = total_features.fillna(method="bfill")
total_features = total_features.dropna(axis='columns')
features = total_features
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features = features.dropna(axis='columns')
total_features = features

# Feature Selection

In [15]:
# Scaling the data
# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

num_features = 25
y = total_features['y']
X_total = total_features.drop(['y', 'index'], axis=1)
print(len(y), len(X_total))

402 402


In [16]:
# Variance Threshold
scaler = MinMaxScaler().fit(X_total)
X_train = scaler.transform(X_total)

total_features_index = []
for feat in X_total:
    total_features_index.append(feat)

X_vt = X_train[0:,0:len(X_total.keys())]

v_threshold = VarianceThreshold(threshold=0.005) # Set a threshold
v_threshold.fit(X_vt)
index = v_threshold.get_support()
true_index = [i for i, x in enumerate(index) if x]
vt_features = [total_features_index[i] for i in true_index]

In [17]:
#Pearson correlation
def cor_selector(X_cor, y,num_feats):
    cor_list = []
    feature_name = X_cor.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X_cor.columns.tolist():
        cor = np.corrcoef(X_cor[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X_cor.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

def pears(X):
    cor_support, cor_feature = cor_selector(X, y, num_features)
    return cor_feature

pc_features = pears(X_total)

In [18]:
#Chi-squared
X_norm = MinMaxScaler().fit_transform(X_total)

chi_selector = SelectKBest(chi2, k=num_features)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = X_total.loc[:,chi_support].columns.tolist()

In [19]:
#Wrapper Forwards
sfs = SFS(svm.SVC(),
          k_features=num_features,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)

sfs.fit(X_total, y)
fwrapper_feature = list(sfs.k_feature_names_)

In [20]:
#Wrapper Backwards
sbs = SFS(svm.SVC(),
         k_features=num_features,
         forward=False,
         floating=False,
         cv=0)
sbs.fit(X_total, y)
bwrapper_feature = list(sbs.k_feature_names_)

In [30]:
#Step-wise wrapper
amount_feat = len(total_features.keys())
left_bound = (amount_feat//2) - (num_features//2)
right_bound =(amount_feat//2) + (num_features//2)

sffs = SFS(svm.SVC(),
         k_features=(left_bound, right_bound),
         forward=True,
         floating=True,
         cv=0)
sffs.fit(X_total, y)
swrapper_feature = list(sffs.k_feature_names_)

In [31]:
# LASSO
num_feats = num_features
scaler = StandardScaler().fit(X_total)
X_train = scaler.transform(X_total)
embeded_lr_selector = SelectFromModel(LogisticRegression(C=1, penalty="l1", solver='liblinear'), max_features=num_feats)
embeded_lr_selector.fit(scaler.transform(X_total), y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X_total.loc[:,embeded_lr_support].columns.tolist()

In [32]:
#Random Forest
scaler = StandardScaler().fit(X_total)
X_train = scaler.transform(X_total)

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_features)
embeded_rf_selector.fit(X_total, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X_total.loc[:,embeded_rf_support].columns.tolist()

In [33]:
#LightBM
lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
embeded_lgb_selector.fit(X_total, y)

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X_total.loc[:,embeded_lgb_support].columns.tolist()

In [34]:
# Lists of features and printing the amount of features selected
# vt_features,
# "variance threshold", 
selected_features = [vt_features, pc_features, chi_feature, fwrapper_feature, bwrapper_feature, swrapper_feature, 
                    embeded_lr_feature, embeded_rf_feature, embeded_lgb_feature]
feature_sel_name = ["variance threshold", "pearson correlation", "chi-squared", "forward wrapper", "backwards wrapper", 
                  "step-wise wrapper", "Lasso", "Random Forest", 'lightGBM']

print(f"total features: {len(total_features.keys())}")
print("Filter:")
for i in range(len(selected_features)):
    print(f"\t {feature_sel_name[i]}: {len(selected_features[i])}")
    if feature_sel_name[i] == "chi-squared":
        print("Wrapper")
    if feature_sel_name[i] == "step-wise wrapper":
        print("Embedded")

total features: 175
Filter:
	 variance threshold: 157
	 pearson correlation: 25
	 chi-squared: 25
Wrapper
	 forward wrapper: 25
	 backwards wrapper: 25
	 step-wise wrapper: 75
Embedded
	 Lasso: 25
	 Random Forest: 25
	 lightGBM: 25


# Classifiers

Logistic Regression

In [35]:
def logistic_func(X_train, X_test, y_train, y_test):
    # define the model
    logistic_model = LogisticRegression(solver='lbfgs', max_iter=100)

    # fit/train the model on all features
    logistic_model.fit(X_train, y_train)
    # Y_test_pred = logistic_model.predict(X_test)

    # # score
    # log_score = logistic_model.score(X_test,y_test)
    return logistic_model #, Y_test_pred, log_score

SVM

In [36]:
def svm_func(X_train, X_test, y_train, y_test):
    # define the model
    svm_model = svm.SVC()

    # fit/train the model on all features
    svm_model.fit(X_train, y_train)
    # Y_test_pred = svm_model.predict(X_test)

    # # score
    # svm_score = svm_model.score(X_test,y_test)
    return svm_model #, Y_test_pred, svm_score

Random Forest

In [37]:
def randomforest_func(X_train, X_test, y_train, y_test):
    # define the model
    randomf_model = RandomForestClassifier()

    # fit/train the model on all features
    randomf_model.fit(X_train, y_train)
    # Y_test_pred = randomf_model.predict(X_test)

    # # score
    # rf_score=randomf_model.score(X_test, y_test)
    return randomf_model #, Y_test_pred, rf_score

Neural Networks

In [46]:
def neuralnetworks_func(X_train, X_test, Y_train, Y_test):
    val_accuracies = []

    ## Convert labels to categorical
    y_train_cat = to_categorical(Y_train, 3)
    y_test_cat = to_categorical(Y_test, 3)

    # Define the feedforward neural network
    neural_model = Sequential()
    neural_model.add(Dense(64, activation='relu'))
    neural_model.add(Dense(64, activation='relu'))
    neural_model.add(Dense(64, activation='relu'))
    neural_model.add(Dense(64, activation = 'relu'))
    neural_model.add(Dropout(0.5))
    neural_model.add(Dense(3, activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    neural_model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

    # Train the model
    # history = model.fit(X_train_pca, y_train, validation_data=(X_test_pca, y_test), epochs=45, batch_size=100, verbose=2)
    history = neural_model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), epochs=45, batch_size=100, verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]
    val_accuracies.append(val_accuracy) # accuracy score

    # print(val_accuracies)
    avg_val_accuracy = sum(val_accuracies) / len(val_accuracies) # average of accurracy scores
    # print("Average Validation Accuracy:", avg_val_accuracy)

    # plt.plot(history.history['accuracy'])
    # plt.plot(history.history['val_accuracy'])
    # plt.legend(['accuracy', 'val_accuracy'])
    # print(neural_model.predict(X_test))
    return val_accuracies


Adaboost

In [39]:
# Import the AdaBoost classifier
def adaboost_func(X_train, X_test, Y_train, Y_test):

        # Create adaboost classifer object
    abc= AdaBoostClassifier(n_estimators=50, learning_rate=1, random_state=5)

    # Train Adaboost Classifer
    adaboost_model = abc.fit(X_train, Y_train)

    return adaboost_model
    #Predict the response for test dataset
    # y_pred = model1.predict(X_test)
        # score30= adaboost_model.score(X_test, y_test)
        # Ada_scores[i] = score30
    # print(Ada_scores)
    # print(np.mean(Ada_scores))
    #predictions = adaboost_model.predict(X_test)
    #print(predictions != y_test)
    #print(score30)



In [31]:
# scores_ada = []
# trainscores_ada = []
# for i in range (50):
#       dtree = DecisionTreeClassifier(criterion='entropy', max_depth=1, random_state=1)
#       adbclassifier = AdaBoostClassifier(base_estimator=dtree,
#                                     n_estimators=100,
#                                     learning_rate=1,
#                                     algorithm = 'SAMME',
#                                     random_state=1)
#       X_train, X_test, y_train, y_test = train_test_split(x_train, y, test_size=0.2, random_state= i+1)
#       adbclassifier.fit(X_train, y_train)
#       scores_ada.append(adbclassifier.score(X_test,y_test))
#       trainscores_ada.append(adbclassifier.score(X_train,y_train))
# print(np.mean(scores_ada))
# print(len(X_test))
# plt.ylim(0,1)
# plt.plot(scores_ada)
# plt.plot(trainscores_ada)

XGBoost

In [40]:
def xgboost_func(X_train, X_test, Y_train, Y_test):
    label = LabelEncoder()
    y2 = label.fit_transform(Y_train)
    y2 = list(y2)

    xgb_clf = XGBClassifier(n_estimators = 500,
                            learning_rate = 1,
                            eval_metric = "logloss",
                            early_stopping_rounds = 5,
                            n_jobs = -1,
                            )
    xgb_clf.fit(X_train, y2,                    
                eval_set = [(X_test,Y_test)],
                verbose = False)
    pred_test = xgb_clf.predict(X_test)
    test_score = accuracy_score(pred_test, Y_test)

    return test_score

# Own measurements

In [58]:
meas_ecgfeatures = pd.DataFrame(None, columns= columns)
meas_edafeatures = None
meas_bvpfeatures = None

meas_fs = 400 # sampling frequency
meas_y = []
index = 0

In [59]:
import os.path
duration=45
meas_subjects = ['Nicky', 'Rick', 'Tomasz', 'Thomas', 'Mack', 'Jack', 'Richelle']
for testday in range(1,3):
    for meas_subject in range(1,8):
        for type_test in ['nultest', 'rest', 'stresstest']:
            go = True
            num_sig = 1
            while go:
                meas_path= r"C:/Users/riche/OneDrive/Documenten/GitHub/EPO4-C2/Testday_" + str(testday) + '/Subject_' + str(meas_subject)
                meas_path_complete = meas_path + '/' + meas_subjects[meas_subject-1] + '-' + type_test + '_' + str(num_sig) + '.csv'
                # print(meas_path_complete)
                check_file = os.path.isfile(meas_path_complete)
                if check_file:
                    print(meas_subjects[meas_subject-1], type_test, num_sig)
                    num_sig += 1
                    meas_data = pd.read_csv(meas_path_complete)
                    meas_ecg_base = meas_data['ECG Data']
                    meas_eda_base = meas_data['GSR Data']
                    meas_t = meas_data['TimeStamp']
                    meas_bvp_base = meas_data['PPG_Red Data']

                    meas_ecg_base = meas_ecg_base[20:]
                    meas_eda_base = meas_eda_base[20:]
                    meas_bvp_base = meas_bvp_base[20:]
                    meas_t = meas_t[20:]
                    print(len(meas_ecg_base))
                    n_samples = duration * meas_fs
                    for j in range(meas_t[len(meas_t)-1]//n_samples): # loop over all segments
                        # Cutting the signal into a segment
                        begin, stop = j*meas_fs*duration, (j+1)*meas_fs*duration
                        
                        start = meas_t.index[meas_t>=begin][0]
                        end = meas_t.index[meas_t<=stop][-1]

                        print(start, end)

                        meas_ecg = meas_ecg_base[start:end]
                        meas_eda = meas_eda_base[start:end]
                        meas_bvp = meas_bvp_base[start:end]
                        
                        # getting the eda and ecg features
                        index += 1

                        tonic, phasic, start, end = get_edaindex(meas_eda, meas_fs)
                        meas_edafeatures = get_edafeatures(index, meas_edafeatures, phasic, tonic, meas_fs)
                        meas_ecgfeatures = get_ecgfeatures(meas_ecg, meas_fs, meas_ecgfeatures, index)
                        
                        bvpfeature = get_bvpfeatures(meas_bvp, index)
                        meas_bvpfeatures = pd.concat([meas_bvpfeatures, bvpfeature], axis=0) 

            

                        if type_test == 'stresstest':
                            meas_y.append(1)
                        else:
                            meas_y.append(0)
                else:
                    go = False
                


Nicky nultest 1
13400
20 7149
Nicky nultest 2
12916
20 7149
Nicky rest 1
95473
20 7150
7151 14362
14362 21573
21574 28785
28785 35996
35997 43208
43208 50419
50420 57631
57631 64843
64843 72054
72055 79266
79266 86477
86478 93689
Nicky stresstest 1
54197
20 7149
7150 14361
14361 21572
21573 28784
28784 35995
35996 43207
43207 50418
Nicky stresstest 2
36158
20 7151
7152 14363
14363 21574
21575 28786
28786 35997
Rick nultest 1
13155
20 7159
Rick nultest 2
12129
20 7156
Rick rest 1
84723
20 7149
7150 14361
14361 21572
21573 28784
28784 35995
35996 43207
43207 50418
50419 57630
57630 64842
64842 72053
72054 79265
Rick stresstest 1
69518
20 7155
7156 14367
14367 21578
21579 28790
28790 36001
36002 43213
43213 50424
50425 57636
57636 64848
Rick stresstest 2
53373
20 7154
7155 14366
14366 21577
21578 28789
28789 36000
36001 43212
43212 50423
Tomasz nultest 1
22472
20 7148
7149 14360
14360 21571
Tomasz nultest 2
17383
20 7152
7153 14364
Tomasz rest 1
36019
20 7139
7140 14351
14351 21562
21563 

In [109]:
meas_bvp_filt = bvp_prep(meas_bvpfeatures)

meas_features1 = pd.merge(meas_ecgfeatures, meas_edafeatures, left_index=True, right_index=True)
meas_features = pd.merge(meas_bvp_filt, meas_features1, on='index')
meas_features['y'] = meas_y
# meas_features = meas_features[meas_features.RSP_RVT != 0.0]
# meas_features

In [112]:
meas_features = meas_features.fillna(method="ffill")
meas_features = meas_features.fillna(method="bfill")
meas_features = meas_features.dropna(axis='columns')
mfeatures = meas_features
mfeatures.replace([np.inf, -np.inf], np.nan, inplace=True)
mfeatures = mfeatures.dropna(axis='columns')
meas_features = mfeatures

In [114]:
meas_features.to_csv("meas_features.csv")

# Train - test split

In [65]:
# function which gives predicted and correct values with missclassifications and scores for rest models
def model_predict(X_test, y_test1, model):
    predictions = model.predict(X_test)
    miss_class = np.where(predictions != y_test1)
    miss_class = miss_class[0]
    score=model.score(X_test, y_test1)
    return score

In [32]:
# def model_predict_neural(X_test, y_test1, model):
#     predictions = model.predict(X_test)
#     predictions = np.argmax(predictions, axis = 1)
#     y_test1 = y_test1.astype('int64')
#     testYarg = np.argmax(y_test1, axis = 1)
#     miss_class = np.where(predictions != testYarg)
#     miss_class = miss_class[0]
#     print("Neural network")
#     print("Predicted:",predictions)
#     print("Correct:  ",testYarg)
#     print("Index missclassified:", miss_class)
#     # print("Score:", val_accuracy, "\n")
#     return predictions, miss_class, testYarg

In [33]:
# y_test_cat = to_categorical(y_test)
# y_test1 = y_test_cat.astype('int64')
# testYarg = np.argmax(y_test1, axis = 1)

NameError: name 'y_test' is not defined

General Approach

In [115]:
subjects = ['S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S13','S14','S15','S16','S17']
model_names = ["LogisticRegression()", "SVC()", "RandomForestClassifier()", "AdaBoostClassifier(learning_rate=1, random_state=5)", "Sequential()",
               "LabelEncoder()"]

first_line = ""
for model in model_names:
    first_line += "\t" + model
print(first_line)

for i in range(len(selected_features)):
    feat = selected_features[i]
    scores_dict = dict()
    for model in model_names:
        scores_dict[model] = []

    for subject in subjects:
        # X_test = total_features[total_features['index'].str.startswith(subject)]
        # y_test = X_test['y']
        # X_test = X_test.drop(['y', 'index'], axis=1)
        X_train = total_features[~total_features['index'].str.startswith(subject)]
        y_train = X_train['y']
        X_train = X_train.drop(['y', 'index'], axis=1)
        # X_train = X_train[list(feat)]
        # X_test = X_test[list(feat)]

        # External testing
        X_test = meas_features
        y_test = X_test['y']
        feat_lst = []
        for feature in feat:
            if feature in X_test.keys():
                feat_lst.append(feature)
        # print(feat_lst)
        X_test = meas_features.drop(['y', 'index'], axis=1)
        X_train = X_train[list(feat_lst)]
        X_test = X_test[list(feat_lst)]
        
        # fit the data
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Get models
        lg_model = logistic_func(X_train, X_test, y_train, y_test)
        svm_model = svm_func(X_train, X_test, y_train, y_test)
        rf_model = randomforest_func(X_train, X_test, y_train, y_test)
        ada_model = adaboost_func(X_train, X_test, y_train, y_test)
        nn_accurary = neuralnetworks_func(X_train, X_test, y_train, y_test)
        xgb_accuracy = xgboost_func(X_train, X_test, y_train, y_test)
        
        # Get the score
        models= [lg_model, svm_model, rf_model, ada_model]

        for model in models:
            score = model_predict(X_test, y_test, model)
            scores_dict[str(model)].append(model_predict(X_test, y_test, model))
        scores_dict["Sequential()"].append(nn_accurary)
        scores_dict["LabelEncoder()"].append(xgb_accuracy)

    line = str(feature_sel_name[i])

    for a in scores_dict:
        score = scores_dict[a]
        line += "\t" + f"{np.mean(score):.5f}" + "+-" + f"{np.std(score):.3f}"

    # line += "\t" + f"{np.mean(nn_accurary):.5f}" + "+-" + f"{np.std(nn_accurary):.3f}"
    print(line)

	LogisticRegression()	SVC()	RandomForestClassifier()	AdaBoostClassifier(learning_rate=1, random_state=5)	Sequential()	LabelEncoder()
variance threshold	0.48594+-0.010	0.48047+-0.000	0.55911+-0.025	0.55312+-0.027	0.48177+-0.005	0.55469+-0.035
pearson correlation	0.49870+-0.020	0.51953+-0.000	0.57943+-0.020	0.56198+-0.035	0.47969+-0.003	0.55833+-0.041
chi-squared	0.48047+-0.000	0.51953+-0.000	0.57708+-0.012	0.57318+-0.016	0.48047+-0.000	0.57734+-0.008
forward wrapper	0.58828+-0.007	0.51068+-0.015	0.58021+-0.009	0.58203+-0.027	0.55729+-0.020	0.62604+-0.031
backwards wrapper	0.49375+-0.003	0.48594+-0.004	0.52995+-0.023	0.54271+-0.023	0.49505+-0.027	0.51875+-0.024
step-wise wrapper	0.49479+-0.022	0.48047+-0.000	0.57031+-0.011	0.55104+-0.032	0.48620+-0.018	0.58099+-0.018
Lasso	0.48047+-0.000	0.51953+-0.000	0.58802+-0.009	0.58177+-0.025	0.48047+-0.000	0.56068+-0.030
Random Forest	0.52969+-0.004	0.48047+-0.000	0.54505+-0.033	0.56016+-0.025	0.48958+-0.025	0.57083+-0.028
lightGBM	0.53203+-0.008	

Subject-based Approach

In [54]:
for i in range(len(selected_features)):
    feat = selected_features[i]
    sb_dict = dict()

    for model in model_names:
        sb_dict[model] = []
        
    for subject in subjects:
        X_sb = total_features[~total_features['index'].str.startswith(subject)]
        y_sb = X_sb['y']
        X_sb = X_sb.drop(['y', 'index'], axis=1)
        X_sb = X_sb[list(feat)]

        X_train_sb, X_test_sb, y_train_sb, y_test_sb = train_test_split(X_sb, y_sb, test_size=0.2, random_state=5)

        scaler_sb = StandardScaler().fit(X_train_sb)
        X_train_sb = scaler_sb.transform(X_train_sb)
        X_test_sb = scaler_sb.transform(X_test_sb)

        lg_model_sb = logistic_func(X_train_sb, X_test_sb, y_train_sb, y_test_sb)
        svm_model_sb = svm_func(X_train_sb, X_test_sb, y_train_sb, y_test_sb)
        rf_model_sb = randomforest_func(X_train_sb, X_test_sb, y_train_sb, y_test_sb)
        # nn_model = neuralnetworks_func(X_train, X_test, Y_train, Y_test)
        ada_model_sb = adaboost_func(X_train_sb, X_test_sb, y_train_sb, y_test_sb)
        nn_accurary_sb = neuralnetworks_func(X_train_sb, X_test_sb, y_train_sb, y_test_sb)
        xgb_accuracy_sb = xgboost_func(X_train_sb, X_test_sb, y_train_sb, y_test_sb)
        # Get the score
        models= [lg_model_sb, svm_model_sb, rf_model_sb, ada_model_sb]

        for model in models:
            score = model_predict(X_test_sb, y_test_sb, model)
            sb_dict[str(model)].append(model_predict(X_test_sb, y_test_sb, model))
        sb_dict["Sequential()"].append(nn_accurary_sb)
        sb_dict["LabelEncoder()"].append(xgb_accuracy_sb)

        # for model in models:
        #     sb_dict[str(model)].append(model_predict(X_test_sb, y_test_sb, model))

    line = ""
    for a in sb_dict:
        line += f"\t {np.mean(sb_dict[a]):.5f}+-{np.std(sb_dict[a]):.3f}"
    print(line)

	 0.95497 +- 0.016	 0.94966 +- 0.020	 0.95233 +- 0.020	 0.94695 +- 0.018	 0.95674 +- 0.022	 0.95411 +- 0.020
	 0.94609 +- 0.018	 0.96289 +- 0.019	 0.94697 +- 0.014	 0.95053 +- 0.021	 0.95763 +- 0.017	 0.94522 +- 0.019
	 0.96290 +- 0.017	 0.96207 +- 0.020	 0.95230 +- 0.020	 0.96022 +- 0.011	 0.96287 +- 0.017	 0.97081 +- 0.012
	 0.93732 +- 0.031	 0.93820 +- 0.021	 0.93285 +- 0.023	 0.90981 +- 0.030	 0.90991 +- 0.029	 0.92399 +- 0.024
	 0.87273 +- 0.029	 0.87986 +- 0.027	 0.87632 +- 0.033	 0.88243 +- 0.020	 0.86041 +- 0.041	 0.87367 +- 0.034
	 0.94167 +- 0.019	 0.95058 +- 0.021	 0.95324 +- 0.025	 0.93553 +- 0.024	 0.94529 +- 0.025	 0.94697 +- 0.018
	 0.97793 +- 0.012	 0.95319 +- 0.022	 0.94439 +- 0.022	 0.94432 +- 0.023	 0.97000 +- 0.019	 0.93725 +- 0.016
	 0.92842 +- 0.020	 0.93462 +- 0.021	 0.96201 +- 0.016	 0.95228 +- 0.022	 0.95323 +- 0.022	 0.96025 +- 0.022
	 0.76574 +- 0.032	 0.79228 +- 0.032	 0.79849 +- 0.033	 0.81173 +- 0.030	 0.80464 +- 0.027	 0.76207 +- 0.031


Hybrid Approach

In [181]:
subjects = ['S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S13','S14','S15','S16','S17']
model_names = ["LogisticRegression()", "SVC()", "RandomForestClassifier()", "AdaBoostClassifier(learning_rate=1, random_state=5)", "Sequential()",
               "LabelEncoder()"]

first_line = ""
for model in model_names:
    first_line += "\t" + model
print(first_line)

y_hy_list= []
amount_per_subject = []
x_trains = np.empty((len(total_features), len(feat)))
total = 0
for subject in subjects:
    X_hy = total_features[total_features['index'].str.startswith(subject)]
    y_hy_list.extend(list(X_hy['y']))
    X_hy = X_hy.drop(['y', 'index'], axis=1)
    X_hy = X_hy[list(feat)]
    scaler_hy = StandardScaler().fit(X_hy)
    x_trains[total:total+len(X_hy),:] = (scaler_hy.transform(X_hy))
    amount_per_subject.append(len(X_hy))
    total+= len(X_hy)

for i in range(len(selected_features)):
    feat = selected_features[i]
    scores_dict = dict()

    for model in model_names:
        scores_dict[model] = []

    start = 0
    for subject in range(len(subjects)):
        X_test = x_trains[start:start+amount_per_subject[subject]]
        y_test = y_hy_list[start:start+amount_per_subject[subject]]
        y_train = []
        if start==0:
            X_train = x_trains[start+amount_per_subject[subject]:]
            y_train = y_hy_list[start+amount_per_subject[subject]:]
        elif start==len(subjects)-1:
            X_train = x_trains[:start]
            y_train = y_hy_list[:start]
        else:
            X_train = np.concatenate((x_trains[:start],x_trains[start + amount_per_subject[subject]:]))
            y_train = y_hy_list[:start] + y_hy_list[start + amount_per_subject[subject]:]
        
        start += amount_per_subject[subject]
     
        # Get models
        lg_model = logistic_func(X_train, X_test, y_train, y_test)
        svm_model = svm_func(X_train, X_test, y_train, y_test)
        rf_model = randomforest_func(X_train, X_test, y_train, y_test)
        ada_model = adaboost_func(X_train, X_test, y_train, y_test)
        nn_accurary = neuralnetworks_func(X_train, X_test, y_train, y_test)
        xgb_accuracy = xgboost_func(X_train, X_test, y_train, y_test)
        
        # Get the score
        models= [lg_model, svm_model, rf_model, ada_model]

        for model in models:
            score = model_predict(X_test, y_test, model)
            scores_dict[str(model)].append(model_predict(X_test, y_test, model))
        scores_dict["Sequential()"].append(nn_accurary)
        scores_dict["LabelEncoder()"].append(xgb_accuracy)

    line = str(feature_sel_name[i])

    for a in scores_dict:
        score = scores_dict[a]
        line += "\t" + f"{np.mean(score):.5f}" + "+-" + f"{np.std(score):.3f}"

    # line += "\t" + f"{np.mean(nn_accurary):.5f}" + "+-" + f"{np.std(nn_accurary):.3f}"
    print(line)

	LogisticRegression()	SVC()	RandomForestClassifier()	AdaBoostClassifier(learning_rate=1, random_state=5)	Sequential()	LabelEncoder()
variance threshold	0.96994+-0.074	0.97701+-0.064	0.97361+-0.050	0.96390+-0.054	0.97454+-0.064	0.96351+-0.064
pearson correlation	0.96994+-0.074	0.97701+-0.064	0.96864+-0.060	0.96390+-0.054	0.96921+-0.069	0.96351+-0.064
chi-squared	0.96994+-0.074	0.97701+-0.064	0.97114+-0.050	0.96390+-0.054	0.95984+-0.073	0.96351+-0.064
forward wrapper	0.96994+-0.074	0.97701+-0.064	0.97361+-0.054	0.96390+-0.054	0.97207+-0.065	0.96351+-0.064
backwards wrapper	0.96994+-0.074	0.97701+-0.064	0.97114+-0.050	0.96390+-0.054	0.96251+-0.088	0.96351+-0.064
step-wise wrapper	0.96994+-0.074	0.97701+-0.064	0.97094+-0.054	0.96390+-0.054	0.97168+-0.069	0.96351+-0.064
Lasso	0.96994+-0.074	0.97701+-0.064	0.97094+-0.054	0.96390+-0.054	0.96728+-0.079	0.96351+-0.064
Random Forest	0.96994+-0.074	0.97701+-0.064	0.97361+-0.050	0.96390+-0.054	0.96444+-0.078	0.96351+-0.064
lightGBM	0.96994+-0.074	